In [2]:
import pandas as pd
import numpy as np
import S2_0_Loading_Data

# Imports: CNN
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input, MaxPooling2D, ZeroPadding2D, Conv2D, Flatten, BatchNormalization, AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.callbacks import EarlyStopping

In [3]:
train_x, test_x, train_y, test_y = S2_0_Loading_Data.load_data()
train_x.shape

(3257, 50, 50, 3)

### We start out with a fairly simple CNN

50x50 faces
Fairly standard CNN architecture. It nets us roughly 78% accuracy.

In [4]:
cnn = Sequential()

cnn.add(ZeroPadding2D(padding = (1, 1), input_shape=(50, 50, 3)))
cnn.add(Conv2D(filters=5, kernel_size=(5, 5), activation="relu"))
cnn.add(Conv2D(filters=5, kernel_size=(5, 5), activation="relu"))
cnn.add(MaxPooling2D(pool_size=(2, 2)))
cnn.add(BatchNormalization())

cnn.add(ZeroPadding2D(padding = (1, 1)))
cnn.add(Conv2D(filters=7, kernel_size=(5, 5), activation="relu"))
cnn.add(Conv2D(filters=7, kernel_size=(5, 5), activation="relu"))
cnn.add(MaxPooling2D(pool_size=(2, 2)))
cnn.add(BatchNormalization())

cnn.add(Flatten())
cnn.add(Dense(49, activation="relu"))
cnn.add(Dropout(0.1))

cnn.add(Dense(3, activation="softmax"))


cnn.compile(
    loss = categorical_crossentropy,
    metrics = ["accuracy"],
    optimizer = "adam"
)

In [5]:
cnn_hist = cnn.fit(
    train_x,
    train_y,
    batch_size = 2000,
    validation_data = (test_x, test_y),
    epochs = 5,
    verbose = 1
)

Epoch 1/5
2/2 [==============================] - 3s 1s/step - loss: 0.8860 - accuracy: 0.6015 - val_loss: 3.3393 - val_accuracy: 0.5252
Epoch 2/5
2/2 [==============================] - 2s 1s/step - loss: 0.5451 - accuracy: 0.7881 - val_loss: 2.0070 - val_accuracy: 0.7914
Epoch 3/5
2/2 [==============================] - 2s 1s/step - loss: 0.4586 - accuracy: 0.8213 - val_loss: 2.2670 - val_accuracy: 0.7975
Epoch 4/5
2/2 [==============================] - 2s 984ms/step - loss: 0.3976 - accuracy: 0.8440 - val_loss: 2.3445 - val_accuracy: 0.7963
Epoch 5/5
2/2 [==============================] - 2s 1s/step - loss: 0.3600 - accuracy: 0.8664 - val_loss: 2.1449 - val_accuracy: 0.7963


### Now, we are trying a more complex CNN

We also scale the images up in attempt to retain more information from the high-resolution images

In [7]:
train_x, test_x, train_y, test_y = S2_0_Loading_Data.load_data(image_size=(100, 100))
train_x.shape

(3257, 100, 100, 3)

In [8]:
model = Sequential(
    [
        Input(shape=(100, 100, 3)),
        Conv2D(10, kernel_size=(3, 3), activation="relu"),
        Conv2D(10, kernel_size=(3, 3), activation="relu"),
        MaxPooling2D(pool_size=(3, 3)),
        Dropout(0.1),
        Conv2D(10, kernel_size=(3, 3), activation="relu"),
        Conv2D(10, kernel_size=(3, 3), activation="relu"),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.1),
        Flatten(),
        Dropout(0.1),
        Dense(256, activation = "relu"),
        Dropout(0.1),
        Dense(3, activation="softmax"),
    ]
)

We can significantly improve our accuracy by using a more advanced architecture and a few changed parameters

In [9]:

model.compile(
    optimizer='rmsprop',
    loss='CategoricalCrossentropy',
    metrics=['accuracy']
)

callback = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)

model_hist = model.fit(
    train_x,
    train_y,
    validation_data = (test_x, test_y),
    epochs = 10,
    verbose = 1,
    callbacks = [callback]
)

Epoch 1/10
102/102 [==============================] - 10s 95ms/step - loss: 1.3650 - accuracy: 0.8327 - val_loss: 0.2391 - val_accuracy: 0.9411
Epoch 2/10
102/102 [==============================] - 10s 96ms/step - loss: 0.2889 - accuracy: 0.9128 - val_loss: 0.2170 - val_accuracy: 0.9472
Epoch 3/10
102/102 [==============================] - 9s 93ms/step - loss: 0.2352 - accuracy: 0.9306 - val_loss: 0.2062 - val_accuracy: 0.9337
Epoch 4/10
102/102 [==============================] - 9s 92ms/step - loss: 0.2061 - accuracy: 0.9407 - val_loss: 0.2248 - val_accuracy: 0.9362
Epoch 5/10
102/102 [==============================] - 10s 100ms/step - loss: 0.1980 - accuracy: 0.9401 - val_loss: 0.1738 - val_accuracy: 0.9448
Epoch 6/10
102/102 [==============================] - 12s 115ms/step - loss: 0.1778 - accuracy: 0.9457 - val_loss: 0.2316 - val_accuracy: 0.9350
Epoch 7/10
102/102 [==============================] - 12s 117ms/step - loss: 0.1637 - accuracy: 0.9460 - val_loss: 0.2023 - val_accuracy

Overall, we get excellent results for CNN

In [11]:
import S4_0_Helper_Functions


preds = model.predict(
    test_x
)

S4_0_Helper_Functions.getAccuracyMetrics(preds, test_y)

accuracy: 0.9558282208588957
recall: 0.9558282208588957
precision: 0.9558282208588957
